In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('twitter-sentiment').getOrCreate()
spark.sparkContext.addPyFile("/home/jovyan/work/sentiment_model.py")
print("Spark context started")

Spark context started


In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType

schema = StructType([
    StructField("target", IntegerType(), True),
    StructField("id", LongType(), True),
    StructField("raw_timestamp", StringType(), True),
    StructField("query_status", StringType(), True),
    StructField("author", StringType(), True),
    StructField("tweet", StringType(), True)
])
    
data_path = "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

raw_sentiment = spark.read.csv(data_path,header=False,schema=schema) \
    .selectExpr("(case when target=4 then 1 else 0 end) as target","tweet")



raw_sentiment.groupBy("target").count().show()

+------+------+
|target| count|
+------+------+
|     1|800000|
|     0|800000|
+------+------+



In [8]:
import pickle as pkl

def read_model(model_path):
    with open(model_path,'rb') as buffer:
        return pkl.load(buffer)

model_path = "/home/jovyan/tweet_sentiment.mdl"

model_object = read_model(model_path)
model_object

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.75, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function pre...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [11]:
model_object_broadcast = spark.sparkContext.broadcast(model_object)

def block_iterator(iterator, size):
    bucket = list()
    for e in iterator:
        bucket.append(e)
        if len(bucket) >= size:
            yield bucket
            bucket = list()
    if bucket:
        yield bucket

def block_classify(iterator):
    model = model_object_broadcast.value
    for features in block_iterator(iterator, 10000):
        import pandas as pd
        import json
        features_df = pd.DataFrame(list(features), columns=["target","text"])
        pred = model.predict_proba(features_df["text"])
        res_df = features_df
        res_df["proba"] = pred[:,1]
        for e in json.loads(res_df.to_json(orient='records')):
            yield e

predicted_df = raw_sentiment.rdd.mapPartitions(block_classify).toDF()

predicted_df.show()

/usr/local/spark/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+------------+------+--------------------+
|       proba|target|                text|
+------------+------+--------------------+
|0.5134324485|     0|@switchfoot http:...|
|0.4883034084|     0|is upset that he ...|
| 0.479001187|     0|@Kenichan I dived...|
| 0.468310149|     0|my whole body fee...|
|0.4856440795|     0|@nationwideclass ...|
|0.4959264044|     0|@Kwesidei not the...|
|0.4829489093|     0|         Need a hug |
|0.4968608392|     0|@LOLTrish hey  lo...|
|0.4830092384|     0|@Tatiana_K nope t...|
|0.4959264044|     0|@twittera que me ...|
|0.5010254401|     0|spring break in p...|
|0.4959264044|     0|I just re-pierced...|
|0.4900247464|     0|@caregiving I cou...|
|0.4686489648|     0|@octolinz16 It it...|
|0.4703860344|     0|@smarrison i woul...|
|0.4789244884|     0|@iamjazzyfizzle I...|
|0.4597403691|     0|Hollis' death sce...|
|0.4977367909|     0|about to file taxes |
|0.4983091373|     0|@LettyA ahh ive a...|
|0.4944417277|     0|@FakerPattyPattz ...|
+----------

In [ ]:
spark.stop()